# NFL Feature Store

1. Config
2. Find data sources
3. Collect data sources
4. Create features


## 1. Config

In [224]:
import pandas as pd
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 1000)
import requests


import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np
import optuna
import xgboost as xgb
from decimal import Decimal
from typing import Any
import datetime

NFLVERSEGITHUB = "https://github.com/nflverse/nflverse-data/releases/download/"
NFL_PLAYER_URL = f"{NFLVERSEGITHUB}players/players.parquet"
NFL_PLAYER_STATS_URL = f"{NFLVERSEGITHUB}player_stats/player_stats.parquet"
NFL_PBP = NFLVERSEGITHUB + "pbp/play_by_play_{season}.parquet"

## Position Configs

### QBs

## ATTRIBUTES

## PARAMETERS

year = 2023

## 2. Find data sources

NFLDATAVERSE: https://github.com/nflverse



## 3. Collect data sources

In [228]:
player_df = pd.read_parquet(NFL_PLAYER_URL)
player_stats_df = pd.read_parquet(NFL_PLAYER_STATS_URL)

## 4. Create Features

In [225]:
player_cols_raw = [
    'display_name', 
    'gsis_id', 
    #'first_name', 
    #'last_name', 
    #'esb_id', 
    'status', 
    'birth_date', 
    'college_name', 
    'position_group', 
    'position', 
    #'jersey_number', 
    'height', 
    'weight', 
    #'years_of_experience', 
    #'team_abbr', 
    #'team_seq', 
    #'current_team_id', 
    #'football_name', 
    'entry_year', 
    #'rookie_year',
    #'draft_club', 
    'draft_number', 
    'college_conference', 
    #'status_description_abbr', 
    #'status_short_description', 
    #'gsis_it_id', 
    #'short_name', 
    #'smart_id', 
    'headshot', 
    #'suffix', 
    #'uniform_number', 
    #'draft_round', 
    #'season'
]

PLAYER_BOXSCORE_COLUMNS = [
    'player_id',
    'season',
    'completions',
    'attempts',
    'passing_yards',
    'passing_tds',
    'interceptions',
    'sacks',
    'sack_yards',
    'sack_fumbles',
    'sack_fumbles_lost',
    'passing_air_yards',
    'passing_yards_after_catch',
    'passing_first_downs',
    'passing_epa',
    'passing_2pt_conversions',
    'pacr',
    'dakota',
    'carries',
    'rushing_yards',
    'rushing_tds',
    'rushing_fumbles',
    'rushing_fumbles_lost',
    'rushing_first_downs',
    'rushing_epa',
    'rushing_2pt_conversions',
    'receptions',
    'targets',
    'receiving_yards',
    'receiving_tds',
    'receiving_fumbles',
    'receiving_fumbles_lost',
    'receiving_air_yards',
    'receiving_yards_after_catch',
    'receiving_first_downs',
    'receiving_epa',
    'receiving_2pt_conversions',
    'racr',
    'target_share',
    'air_yards_share',
    'wopr',
    'special_teams_tds',
    'fantasy_points',
    'fantasy_points_ppr'
]

PLAYER_POINTS_COLUMNS = [
    'player_id',
    'season',
    'fantasy_points',
    'fantasy_points_ppr'
]

FEATURE_STORE_METADATA = [
    {"name": "completions", "friendly_name": "Completions", "description": "The number of completed passes.", "dtype": np.int32, "type_group": "Passing", "selectable": False},
    {"name": "attempts", "friendly_name": "Attempts", "description": "The number of pass attempts as defined by the NFL.", "dtype": np.int32, "type_group": "Passing", "selectable": True},
    {"name": "passing_yards", "friendly_name": "Passing Yards", "description": "Yards gained on pass plays.", "dtype": np.int32, "type_group": "Passing", "selectable": True},
    {"name": "passing_tds", "friendly_name": "Passing Touchdowns", "description": "The number of passing touchdowns.", "dtype": np.int32, "type_group": "Passing", "selectable": True},
    {"name": "interceptions", "friendly_name": "Interceptions", "description": "The number of interceptions thrown.", "dtype": np.int32, "type_group": "Passing", "selectable": True},
    {"name": "sacks", "friendly_name": "Sacks", "description": "The Number of times sacked.", "dtype": np.int32, "type_group": "Passing", "selectable": True},
    {"name": "sack_yards", "friendly_name": "Sack Yards", "description": "Yards lost on sack plays.", "dtype": np.int32, "type_group": "Passing", "selectable": True},
    {"name": "sack_fumbles", "friendly_name": "Sack Fumbles", "description": "The number of sacks with a fumble.", "dtype": np.int32, "type_group": "Passing", "selectable": True},
    {"name": "sack_fumbles_lost", "friendly_name": "Sack Fumbles Lost", "description": "The number of sacks with a lost fumble.", "dtype": np.int32, "type_group": "Passing", "selectable": True},
    {"name": "passing_air_yards", "friendly_name": "Passing Air Yards", "description": "Passing air yards (includes incomplete passes).", "dtype": np.int32, "type_group": "Passing", "selectable": True},
    {"name": "passing_yards_after_catch", "friendly_name": "Passing Yards After Catch", "description": "Yards after the catch gained on plays in which player was the passer (this is an unofficial stat and may differ slightly between different sources).", "dtype": np.int32, "type_group": "Passing", "selectable": True},
    {"name": "passing_first_downs", "friendly_name": "Passing First Downs", "description": "First downs on pass attempts.", "dtype": np.int32, "type_group": "Passing", "selectable": True},
    {"name": "passing_epa", "friendly_name": "Passing EPA", "description": "Total expected points added on pass attempts and sacks. NOTE: this uses the variable qb_epa, which gives QB credit for EPA for up to the point where a receiver lost a fumble after a completed catch and makes EPA work more like passing yards on plays with fumbles.", "dtype": np.float32, "type_group": "Passing", "selectable": True},
    {"name": "passing_2pt_conversions", "friendly_name": "Passing 2pt Conversions", "description": "Two-point conversion passes.", "dtype": np.int32, "type_group": "Passing", "selectable": True},
    {"name": "pacr", "friendly_name": "PACR", "description": "Passing Air Conversion Ratio. PACR = passing_yards / passing_air_yards", "dtype": np.float32, "type_group": "Passing", "selectable": True},
    {"name": "dakota", "friendly_name": "Dakota", "description": "Dakota rating for passing efficiency.", "dtype": np.float32, "type_group": "Passing", "selectable": True},

    {"name": "carries", "friendly_name": "Carries", "description": "The number of official rush attempts (incl. scrambles and kneel downs). Rushes after a lateral reception don't count as carry.", "dtype": np.int32, "type_group": "Rushing", "selectable": True},
    {"name": "rushing_yards", "friendly_name": "Rushing Yards", "description": "Yards gained when rushing with the ball (incl. scrambles and kneel downs). Also includes yards gained after obtaining a lateral on a play that started with a rushing attempt.", "dtype": np.int32, "type_group": "Rushing", "selectable": True},
    {"name": "rushing_tds", "friendly_name": "Rushing Touchdowns", "description": "The number of rushing touchdowns (incl. scrambles). Also includes touchdowns after obtaining a lateral on a play that started with a rushing attempt.", "dtype": np.int32, "type_group": "Rushing", "selectable": True},
    {"name": "rushing_fumbles", "friendly_name": "Rushing Fumbles", "description": "The number of rushes with a fumble.", "dtype": np.int32, "type_group": "Rushing", "selectable": True},
    {"name": "rushing_fumbles_lost", "friendly_name": "Rushing Fumbles Lost", "description": "The number of rushes with a lost fumble.", "dtype": np.int32, "type_group": "Rushing", "selectable": True},
    {"name": "rushing_first_downs", "friendly_name": "Rushing First Downs", "description": "First downs on rush attempts (incl. scrambles).", "dtype": np.int32, "type_group": "Rushing", "selectable": True},
    {"name": "rushing_epa", "friendly_name": "Rushing EPA", "description": "Expected points added on rush attempts (incl. scrambles and kneel downs).", "dtype": np.float32, "type_group": "Rushing", "selectable": True},
    {"name": "rushing_2pt_conversions", "friendly_name": "Rushing 2pt Conversions", "description": "Two-point conversion rushes.", "dtype": np.int32, "type_group": "Rushing", "selectable": True},

    {"name": "receptions", "friendly_name": "Receptions", "description": "The number of pass receptions. Lateral receptions officially don't count as reception.", "dtype": np.int32, "type_group": "Receiving", "selectable": True},
    {"name": "targets", "friendly_name": "Targets", "description": "The number of pass plays where the player was the targeted receiver.", "dtype": np.int32, "type_group": "Receiving", "selectable": True},
    {"name": "receiving_yards", "friendly_name": "Receiving Yards", "description": "Yards gained after a pass reception. Includes yards gained after receiving a lateral on a play that started as a pass play.", "dtype": np.int32, "type_group": "Receiving", "selectable": True},
    {"name": "receiving_tds", "friendly_name": "Receiving Touchdowns", "description": "The number of touchdowns following a pass reception. Also includes touchdowns after receiving a lateral on a play that started as a pass play.", "dtype": np.int32, "type_group": "Receiving", "selectable": True},
    {"name": "receiving_air_yards", "friendly_name": "Receiving Air Yards", "description": "Receiving air yards (incl. incomplete passes).", "dtype": np.int32, "type_group": "Receiving", "selectable": True},
    {"name": "receiving_yards_after_catch", "friendly_name": "Receiving Yards After Catch", "description": "Yards after the catch gained on plays in which player was receiver (this is an unofficial stat and may differ slightly between different sources).", "dtype": np.int32, "type_group": "Receiving", "selectable": True},
    {"name": "receiving_first_downs", "friendly_name": "Receiving First Downs", "description": "First downs by reception.", "dtype": np.int32, "type_group": "Receiving", "selectable": True},
    {"name": "receiving_epa", "friendly_name": "Receiving EPA", "description": "The expected points added by receiving.", "dtype": np.float32, "type_group": "Receiving", "selectable": True},
    {"name": "receiving_fumbles", "friendly_name": "Receiving Fumbles", "description": "The number of fumbles after a pass reception.", "dtype": np.int32, "type_group": "Receiving", "selectable": True},
    {"name": "receiving_fumbles_lost", "friendly_name": "Receiving Fumbles Lost", "description": "The number of fumbles lost after a pass reception.", "dtype": np.int32, "type_group": "Receiving", "selectable": True},
    {"name": "receiving_2pt_conversions", "friendly_name": "Receiving 2pt Conversions", "description": "Two-point conversion receptions.", "dtype": np.int32, "type_group": "Receiving", "selectable": True},
    {"name": "racr", "friendly_name": "RACR", "description": "Receiver Air Conversion Ratio. RACR = receiving_yards / receiving_air_yards.", "dtype": np.float32, "type_group": "Receiving", "selectable": True},
    {"name": "target_share", "friendly_name": "Target Share", "description": "The share of targets of the player in all targets of his team.", "dtype": np.float32, "type_group": "Receiving", "selectable": True},
    {"name": "air_yards_share", "friendly_name": "Air Yards Share", "description": "The share of receiving_air_yards of the player in all air_yards of his team.", "dtype": np.float32, "type_group": "Receiving", "selectable": True},
    {"name": "wopr", "friendly_name": "WOPR", "description": "Weighted Opportunity Rating. WOPR = 1.5 × target_share + 0.7 × air_yards_share.", "dtype": np.float32, "type_group": "Receiving", "selectable": True},

    {"name": "special_teams_tds", "friendly_name": "Special Teams Touchdowns", "description": "The number of touchdowns scored on special teams.", "dtype": np.int32, "type_group": "SpecialTeams", "selectable": True},

    {"name": "player_id", "friendly_name": "Player ID", "description": "ID of the player. Use this to join to other sources.", "dtype": str, "type_group": "Identifier", "selectable": False},
    {"name": "display_name", "friendly_name": "Display Name", "description": "The display name of the player.", "dtype": str, "type_group": "Identifier", "selectable": False},
    {"name": "status", "friendly_name": "Status", "description": "Player Status (ACT, RET, CUT)", "dtype": str, "type_group": "Identifier", "selectable": False},
    {"name": "position", "friendly_name": "Position", "description": "Position of the player.", "dtype": str, "type_group": "Identifier", "selectable": False},
    {"name": "position_group", "friendly_name": "Position Group", "description": "High Level Position Group of the player.", "dtype": str, "type_group": "Identifier", "selectable": False},

    {"name": "season", "friendly_name": "Season", "description": "The season year.", "dtype": np.int32, "type_group": "Identifier", "selectable": True},
    {"name":"games_played", "friendly_name": "Games Played", "description": "Games the player played in that season", "dtype": np.int32, "type_group": "Identifier", "selectable": False},
    {"name": "entry_year", "friendly_name": "Entry Year", "description": "The year the player entered the league.", "dtype": np.int32, "type_group": "Identifier", "selectable": True},
    {"name": "birth_date", "friendly_name": "Birth Date", "description": "The birth date of the player.", "dtype": str, "type_group": "Identifier", "selectable": False},
    {"name": "draft_number", "friendly_name": "Draft Number", "description": "The draft number of the player.", "dtype": np.int32, "type_group": "Identifier", "selectable": True},
    {"name": "college_name", "friendly_name": "College Name", "description": "The college the player attended.", "dtype": str, "type_group": "Identifier", "selectable": False},
    {"name": "college_conference", "friendly_name": "College Conference", "description": "The conference the college is a part of", "dtype": str, "type_group": "Identifier", "selectable": False},
    {"name": "years_of_experience", "friendly_name": "Years of Experience", "description": "Number of years in the league", "dtype": np.int32, "type_group": "Identifier", "selectable": False},

    {"name": "height", "friendly_name": "Height", "description": "The height of the player.", "dtype": np.float32, "type_group": "Attribute", "selectable": True},
    {"name": "weight", "friendly_name": "Weight", "description": "The weight of the player.", "dtype": np.float32, "type_group": "Attribute", "selectable": True},

    {"name": "fantasy_points", "friendly_name": "Fantasy Points", "description": "Standard fantasy points.", "dtype": np.float32, "type_group": "Target", "selectable": False},
    {"name": "fantasy_points_ppr", "friendly_name": "Fantasy Points PPR", "description": "PPR fantasy points.", "dtype": np.float32, "type_group": "Target", "selectable": False},
    {"name": "position_rank", "friendly_name": "Position Rank", "description": "Fantasy Rank by Position Group", "dtype": np.int32, "type_group": "Target", "selectable": False},
    {"name": "ppr_position_rank", "friendly_name": "PPR Position Rank", "description": "PPR Fantasy Rank by Position Group", "dtype": np.int32, "type_group": "Target", "selectable": False},
]

def get_fs_meta_dict():
    return {i["name"]:i["dtype"] for i in FEATURE_STORE_METADATA}

def get_fs_cols(cols, method='contains'):
    fs_meta_dict = get_fs_meta_dict()
    fs_meta_keys = fs_meta_dict.keys()
    fs_cols = {}
    if method == 'contains':
        for col in cols:
            for fs_key in fs_meta_keys:
                if fs_key in col and 'avg_' not in col:
                    fs_cols[col] = fs_meta_dict[fs_key]
    return fs_cols
def _cast2date(value: Any) -> Any:
    if isinstance(value, float) and (np.isnan(value) or np.isinf(value)):
        return None
    if pd.isna(value) or value is None:
        return None
    if isinstance(value, datetime.date):
        return value
    return pd.to_datetime(value).date()

def _cast_pandas_column(df: pd.DataFrame, col: str, current_type: str, desired_type: str) -> pd.DataFrame:
    if desired_type == "datetime64":
        df[col] = pd.to_datetime(df[col])
    elif desired_type == "date":
        df[col] = df[col].apply(lambda x: _cast2date(value=x)).replace(to_replace={pd.NaT: None})
    elif desired_type == "bytes":
        df[col] = df[col].astype("string").str.encode(encoding="utf-8").replace(to_replace={pd.NA: None})
    elif desired_type == "decimal":
        # First cast to string
        df = _cast_pandas_column(df=df, col=col, current_type=current_type, desired_type="string")
        # Then cast to decimal
        df[col] = df[col].apply(lambda x: Decimal(str(x)) if str(x) not in ("", "none", "None", " ", "<NA>") else None)
    else:
        try:
            df[col] = df[col].astype(desired_type)
        except TypeError as ex:
            if "object cannot be converted to an IntegerDtype" not in str(ex):
                raise ex
            df[col] = (
                df[col]
                .apply(lambda x: int(x) if str(x) not in ("", "none", "None", " ", "<NA>") else None)
                .astype(desired_type)
            )
    return df

def fs_apply_type(df, method='equals'):
    fs_meta_dict = get_fs_meta_dict()
    fs_meta_keys = fs_meta_dict.keys()
    for column in df.columns:
        if method == ' equals':
            if column in fs_meta_keys:
                df[column] = df[column].astype(fs_meta_dict[column])
        elif method == 'contains':
            for fs_key in fs_meta_keys:
                if fs_key in column and 'avg_' not in column:
                    df[column] = df[column].astype(fs_meta_dict[fs_key])
    return df


def etl_player_df():
    player_df = pd.read_parquet(NFL_PLAYER_URL)
    player_df = player_df[player_cols_raw]
    player_df = player_df[player_df.position_group.isin(['QB', 'RB', 'WR', 'TE'])].copy()
    player_df = player_df.rename(columns={'gsis_id':'player_id'})
    #player_df = player_df.drop(columns=['position','position_group'])
    return player_df
    #player_df = player_df.fillna(-1)
    #return fs_apply_type(player_df)

    
def etl_player_info_df():
    player_stats_df = pd.read_parquet(NFL_PLAYER_STATS_URL)
    player_stats_df = player_stats_df[player_stats_df.position_group.isin(['QB', 'RB', 'WR', 'TE'])].copy()
    player_stats_df = player_stats_df[player_stats_df.season_type == 'REG'].drop(columns=['player_name','player_display_name','headshot_url','recent_team','season_type','opponent_team']).copy()
    player_stats_df['games_played'] = 1
    return player_stats_df
    #player_stats_df = player_stats_df.fillna(-1)
    #return fs_apply_type(player_stats_df)

def target_feature_store(player_weekly_stats_df):
    position_group_df = player_weekly_stats_df.drop_duplicates(['player_id'])[['player_id','position_group']]

    player_points_df = player_weekly_stats_df[PLAYER_POINTS_COLUMNS].groupby(['player_id', 'season'])['fantasy_points'].sum().reset_index()
    player_points_df['fantasy_points_ppr'] = player_weekly_stats_df[PLAYER_POINTS_COLUMNS].groupby(['player_id', 'season'])['fantasy_points_ppr'].sum().reset_index()['fantasy_points_ppr']
    
    player_points_df = pd.merge(player_points_df, position_group_df, on=['player_id'], how='left')
    player_points_df['position_rank'] = player_points_df.groupby(['position_group', 'season'])['fantasy_points'].rank(ascending=False, method='first')
    player_points_df['position_rank'] = player_points_df['position_rank'].astype(np.int32)
    player_points_df['ppr_position_rank'] = player_points_df.groupby(['position_group', 'season'])['fantasy_points_ppr'].rank(ascending=False, method='first')
    player_points_df['ppr_position_rank'] = player_points_df['ppr_position_rank'].astype(np.int32)
    player_points_df = player_points_df.drop(columns=['position_group'])

    #player_points_df['season'] = player_points_df['season'] + 1
    return player_points_df
    
def season_total_feature_store(player_weekly_stats_df):
    # Group by player and season to get total stats
    position_group_df = player_weekly_stats_df.drop_duplicates(['player_id'])[['player_id','position_group']]
    total_stats = player_weekly_stats_df[PLAYER_BOXSCORE_COLUMNS+['games_played']].groupby(['player_id', 'season']).sum().reset_index()
    total_stats = total_stats[(total_stats.games_played > 3)].copy()
    player_info_df = pd.DataFrame()
    for n_last in [0, 1, 2]:
        t_copy = total_stats.copy()
        prefix ='last_year_' if n_last == 0 else f'{n_last+1}_years_ago_'
        t_copy['season'] = t_copy['season'] + n_last
        t_copy = pd.merge(t_copy, position_group_df, on=['player_id'], how='left')
        t_copy['position_rank'] = t_copy.groupby(['position_group', 'season'])['fantasy_points'].rank(ascending=False, method='first')
        t_copy['position_rank'] = t_copy['position_rank'].astype(np.int32)
        t_copy['ppr_position_rank'] = t_copy.groupby(['position_group', 'season'])['fantasy_points_ppr'].rank(ascending=False, method='first')
        t_copy['ppr_position_rank'] = t_copy['ppr_position_rank'].astype(np.int32)
        t_copy = t_copy.drop(columns=['position_group'])
        t_copy.columns = [f'total_{prefix}{col}' if col not in ['player_id', 'season'] else col for col in t_copy.columns]
        join_cols = ['player_id', 'season'] if 'season' in player_info_df.columns else ['player_id']
        if player_info_df.shape[0] != 0:
            player_info_df = pd.merge(player_info_df, t_copy, on=join_cols, how='left')
        else:
            player_info_df = t_copy
    t_copy = total_stats.copy()
    t_copy = t_copy.sort_values(by=['player_id', 'season'])
    #t_copy['season'] = t_copy['season'] + 1
    t_copy.columns = [f'total_career_{col}' if col not in ['player_id', 'season'] else col for col in t_copy.columns]
    feature_cols = [i for i in t_copy.columns if i not in ['player_id', 'season']]
    t_copy[feature_cols] = t_copy.groupby('player_id')[feature_cols].cumsum()
    player_info_df = pd.merge(player_info_df, t_copy, on=join_cols, how='left')

    del t_copy
    schema = get_fs_cols(player_info_df.columns)
    player_info_df = player_info_df.fillna(-1)
    player_info_df = player_info_df[player_info_df.season != -1].copy()
    player_info_df['season'] = player_info_df['season'] + 1
    player_info_df['season'] = player_info_df['season'].astype(np.int32)
    for column, dtype in schema.items():
        player_info_df[column] = player_info_df[column].astype(dtype)        
    return player_info_df.sort_values(by=['player_id', 'season'])

def season_avg_feature_store(player_weekly_stats_df):
    # Compute average stats per season
    avg_stats = player_weekly_stats_df[PLAYER_BOXSCORE_COLUMNS + ['games_played']].groupby(['player_id', 'season']).sum().reset_index()
    avg_stats = avg_stats[(avg_stats.games_played > 3)].copy()
    stat_columns = avg_stats.columns.difference(['player_id', 'season', 'games_played'])
    avg_stats[stat_columns] = avg_stats[stat_columns].div(avg_stats['games_played'], axis=0)
    avg_stats.drop(columns=['games_played'], inplace=True)

    player_info_df = pd.DataFrame()
    
    # Include last 1, 2, and 3 years averages
    for n_last in [0, 1, 2]:
        t_copy = avg_stats.copy()
        prefix = 'last_year_' if n_last == 0 else f'{n_last + 1}_years_ago_'
        t_copy['season'] = t_copy['season'] + n_last
        t_copy.columns = [f'avg_{prefix}{col}' if col not in ['player_id', 'season'] else col for col in t_copy.columns]
        join_cols = ['player_id', 'season'] if 'season' in player_info_df.columns else ['player_id']
        if player_info_df.shape[0] != 0:
            player_info_df = pd.merge(player_info_df, t_copy, on=join_cols, how='left')
        else:
            player_info_df = t_copy
    
    # Compute cumulative average stats
    t_copy = avg_stats.copy()
    t_copy = t_copy.sort_values(by=['player_id', 'season'])
    t_copy.columns = [f'avg_career_{col}' if col not in ['player_id', 'season'] else col for col in t_copy.columns]
    feature_cols = [i for i in t_copy.columns if i not in ['player_id', 'season']]
    
    for col in feature_cols:
        t_copy[col] = t_copy.groupby('player_id')[col].expanding().mean().reset_index(level=0, drop=True)
    
    # Merge cumulative average stats with player_info_df
    join_cols = ['player_id', 'season']
    player_info_df = pd.merge(player_info_df, t_copy, on=join_cols, how='left')
    
    # Final adjustments
    player_info_df = player_info_df.fillna(-1)
    player_info_df = player_info_df[player_info_df.season != -1].copy()
    player_info_df['season'] = player_info_df['season'] + 1
    player_info_df['season'] = player_info_df['season'].astype(np.int32)
    return player_info_df.sort_values(by=['player_id', 'season'])

def make_season_feature_store():
    player_df = etl_player_df()
    player_weekly_stats_df = etl_player_info_df()
    player_points_df = target_feature_store(player_weekly_stats_df)
    player_season_total_fs_df = season_total_feature_store(player_weekly_stats_df)
    player_season_avg_fs_df = season_avg_feature_store(player_weekly_stats_df)
    fs_df = pd.merge(player_season_total_fs_df, player_season_avg_fs_df, on=['player_id', 'season'], how='left')
    del player_season_total_fs_df, player_season_avg_fs_df
    
    fs_df = pd.merge(fs_df, player_points_df, on=['player_id', 'season'], how='left')
    fs_df = pd.merge(fs_df,player_df, on=['player_id'], how='left')
    
    fs_df['years_of_experience'] = fs_df['season'] - fs_df['entry_year']
    fs_df.fillna(-1, inplace=True)

    # rank by position and season for ranked by total fantasy points and make a column called position_rank

    fs_df = fs_df[(fs_df.fantasy_points > 0) | (fs_df.fantasy_points_ppr > 0) | (fs_df.season == fs_df.season.max())].copy()
    return fs_df
    #return fs_apply_type(fs_df, method='contains')

In [241]:
import os
import pyarrow as pa
from typing import List
def get_dataframe(path: str, columns: List = None):
    """
    Read a DataFrame from a parquet file.

    Args:
        path (str): Path to the parquet file.
        columns (List): List of columns to select (default is None).

    Returns:
        pd.DataFrame: Read DataFrame.
    """
    try:
        return pd.read_parquet(path, engine='pyarrow', dtype_backend='numpy_nullable', columns=columns)
    except Exception as e:
        print(e)
        return pd.DataFrame()
root_path = '../data/feature_store'

fs_type_path = 'player/season'
fs_file_name = 'fs.parquet'
path = f"{root_path}/{fs_type_path}/{fs_file_name}"
fs_df = get_dataframe(path)

,player_id,season,total_last_year_completions,total_last_year_attempts,total_last_year_passing_yards,total_last_year_passing_tds,total_last_year_interceptions,total_last_year_sacks,total_last_year_sack_yards,total_last_year_sack_fumbles,total_last_year_sack_fumbles_lost,total_last_year_passing_air_yards,total_last_year_passing_yards_after_catch,total_last_year_passing_first_downs,total_last_year_passing_epa,total_last_year_passing_2pt_conversions,total_last_year_pacr,total_last_year_dakota,total_last_year_carries,total_last_year_rushing_yards,total_last_year_rushing_tds,total_last_year_rushing_fumbles,total_last_year_rushing_fumbles_lost,total_last_year_rushing_first_downs,total_last_year_rushing_epa,total_last_year_rushing_2pt_conversions,total_last_year_receptions,total_last_year_targets,total_last_year_receiving_yards,total_last_year_receiving_tds,total_last_year_receiving_fumbles,total_last_year_receiving_fumbles_lost,total_last_year_receiving_air_yards,total_last_year_receiving_yards_after_catch,total_last_year_receiving_first_downs,total_last_year_receiving_epa,total_last_year_receiving_2pt_conversions,total_last_year_racr,total_last_year_target_share,total_last_year_air_yards_share,total_last_year_wopr,total_last_year_special_teams_tds,total_last_year_fantasy_points,total_last_year_fantasy_points_ppr,total_last_year_games_played,total_last_year_position_rank,total_last_year_ppr_position_rank,total_2_years_ago_completions,total_2_years_ago_attempts,total_2_years_ago_passing_yards,total_2_years_ago_passing_tds,total_2_years_ago_interceptions,total_2_years_ago_sacks,total_2_years_ago_sack_yards,total_2_years_ago_sack_fumbles,total_2_years_ago_sack_fumbles_lost,total_2_years_ago_passing_air_yards,total_2_years_ago_passing_yards_after_catch,total_2_years_ago_passing_first_downs,total_2_years_ago_passing_epa,total_2_years_ago_passing_2pt_conversions,total_2_years_ago_pacr,total_2_years_ago_dakota,total_2_years_ago_carries,total_2_years_ago_rushing_yards,total_2_years_ago_rushing_tds,total_2_years_ago_rushing_fumbles,total_2_years_ago_rushing_fumbles_lost,total_2_years_ago_rushing_first_downs,total_2_years_ago_rushing_epa,total_2_years_ago_rushing_2pt_conversions,total_2_years_ago_receptions,total_2_years_ago_targets,total_2_years_ago_receiving_yards,total_2_years_ago_receiving_tds,total_2_years_ago_receiving_fumbles,total_2_years_ago_receiving_fumbles_lost,total_2_years_ago_receiving_air_yards,total_2_years_ago_receiving_yards_after_catch,total_2_years_ago_receiving_first_downs,total_2_years_ago_receiving_epa,total_2_years_ago_receiving_2pt_conversions,total_2_years_ago_racr,total_2_years_ago_target_share,total_2_years_ago_air_yards_share,total_2_years_ago_wopr,total_2_years_ago_special_teams_tds,total_2_years_ago_fantasy_points,total_2_years_ago_fantasy_points_ppr,total_2_years_ago_games_played,total_2_years_ago_position_rank,total_2_years_ago_ppr_position_rank,total_3_years_ago_completions,total_3_years_ago_attempts,total_3_years_ago_passing_yards,total_3_years_ago_passing_tds,total_3_years_ago_interceptions,total_3_years_ago_sacks,total_3_years_ago_sack_yards,total_3_years_ago_sack_fumbles,...,avg_3_years_ago_attempts,avg_3_years_ago_passing_yards,avg_3_years_ago_passing_tds,avg_3_years_ago_interceptions,avg_3_years_ago_sacks,avg_3_years_ago_sack_yards,avg_3_years_ago_sack_fumbles,avg_3_years_ago_sack_fumbles_lost,avg_3_years_ago_passing_air_yards,avg_3_years_ago_passing_yards_after_catch,avg_3_years_ago_passing_first_downs,avg_3_years_ago_passing_epa,avg_3_years_ago_passing_2pt_conversions,avg_3_years_ago_pacr,avg_3_years_ago_dakota,avg_3_years_ago_carries,avg_3_years_ago_rushing_yards,avg_3_years_ago_rushing_tds,avg_3_years_ago_rushing_fumbles,avg_3_years_ago_rushing_fumbles_lost,avg_3_years_ago_rushing_first_downs,avg_3_years_ago_rushing_epa,avg_3_years_ago_rushing_2pt_conversions,avg_3_years_ago_receptions,avg_3_years_ago_targets,avg_3_years_ago_receiving_yards,avg_3_years_ago_receiving_tds,avg_3_years_ago_receiving_fu

In [212]:
a = fs_df[fs_df.season==2024].copy()
a['total_last_year_position_rank'] = a['total_last_year_position_rank'].astype(int)
a[(a.position_group == 'WR')].sort_values(by=['total_last_year_position_rank'])[['player_id','display_name','position_group','total_last_year_position_rank','total_last_year_fantasy_points','total_last_year_games_played']]

,player_id,display_name,position_group,total_last_year_position_rank,total_last_year_fantasy_points,total_last_year_games_played
10054,00-0036358,CeeDee Lamb,WR,1,270.200012,17
8678,00-0033040,Tyreek Hill,WR,2,257.399994,16
10245,00-0036963,Amon-Ra St. Brown,WR,3,211.899994,16
7975,00-0031408,Mike Evans,WR,4,203.500000,17
10486,00-0039075,Puka Nacua,WR,5,193.500000,17
9520,00-0034827,D.J. Moore,WR,6,190.500000,17
9878,00-0035719,Deebo Samuel,WR,7,183.699997,15
9845,00-0035676,A.J. Brown,WR,8,183.600006,17
10136,00-0036554,Nico Collins,WR,9,180.399994,15
9988,00-0036261,Brandon Aiyuk,WR,10,174.199997,16


In [32]:
player_df = pd.read_csv('https://github.com/dynastyprocess/data/raw/master/files/db_playerids.csv')
player_df

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
0,16579,49d4b627-3f19-49a8-ae33-f539e0fb1f88,23084.0,NaN,NaN,11560.0,NaN,4431611.0,40900.0,NaN,2669577.0,NaN,NaN,17695.0,NaN,1550.0,40900.0,0.0,NaN,1286445.0,Caleb Williams,caleb williams,QB,CHI,2001-11-18,22.6,2024.0,1.0,1.0,NaN,NaN,73.0,215.0,USC,2024
1,16580,d0c0630e-8925-4b81-a32b-e4146d95f01f,23046.0,NaN,NaN,11564.0,NaN,4431452.0,40881.0,NaN,2669575.0,NaN,NaN,17673.0,NaN,1551.0,40881.0,0.0,NaN,1286803.0,Drake Maye,drake maye,QB,NEP,2002-08-30,21.9,2024.0,1.0,3.0,NaN,NaN,76.0,225.0,North Carolina,2024
2,16581,bab8c65a-9fff-410d-8e34-dc2f636d890e,22902.0,NaN,NaN,11566.0,NaN,4426348.0,40896.0,NaN,3123337.0,NaN,NaN,17692.0,NaN,1552.0,40896.0,0.0,NaN,1161228.0,Jayden Daniels,jayden daniels,QB,WAS,2000-12-18,23.6,2024.0,1.0,2.0,NaN,NaN,76.0,210.0,LSU,2024
3,16582,f056c8ed-6ee5-492c-af11-3f7e3d97195c,22910.0,NaN,NaN,11563.0,NaN,4426338.0,40875.0,NaN,3121193.0,NaN,NaN,16995.0,NaN,1553.0,40875.0,0.0,NaN,1161899.0,Bo Nix,bo nix,QB,DEN,2000-02-25,24.4,2024.0,1.0,12.0,NaN,NaN,74.0,217.0,Oregon,2024
4,16583,6499a161-b221-4591-b122-d99ece1be64b,22973.0,NaN,NaN,11559.0,NaN,4360423.0,40889.0,NaN,2963496.0,NaN,NaN,17700.0,NaN,1554.0,40889.0,0.0,NaN,1107478.0,Michael Penix Jr.,michael penix,QB,ATL,2000-05-08,24.2,2024.0,1.0,8.0,NaN,NaN,74.0,216.0,Washington,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11680,972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Doug Brien,doug brien,PK,NOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024
11681,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jeremy Brigham,jeremy brigham,TE,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024
11682,984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vincent Brisby,vincent brisby,WR,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024
11683,990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bubby Brister,bubby brister,QB,KCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024


In [34]:
fp_df.page_type.unique()

fp_df[fp_df.page_type=='best-overall']

array(['best-overall', 'best-dst', 'best-qb', 'best-rb', 'best-te',
       'best-wr', 'dynasty-offense', 'weekly-offense', 'dynasty-idp',
       'dynasty-overall', 'weekly-kdst', 'dynasty-db', 'dynasty-dl',
       'dynasty-dst', 'dynasty-k', 'dynasty-lb', 'dynasty-qb',
       'dynasty-rb', 'dynasty-te', 'dynasty-wr', 'dynasty-rk',
       'dynasty-op', 'redraft-offense', 'redraft-idp', 'redraft-overall',
       'redraft-kdst', 'redraft-db', 'redraft-dl', 'redraft-dst',
       'redraft-k', 'redraft-lb', 'redraft-qb', 'redraft-rb',
       'redraft-te', 'redraft-wr', 'redraft-op', 'weekly-idp',
       'weekly-db', 'weekly-dl', 'weekly-dst', 'weekly-k', 'weekly-lb',
       'weekly-qb', 'weekly-rb', 'weekly-te', 'weekly-wr', 'weekly-op'],
      dtype=object)

In [36]:
fp_df[fp_df.page_type=='weekly-db']

,fp_page,page_type,player,id,pos,team,ecr,sd,best,worst,mergename,tm,sportsdata_id,player_filename,yahoo_id,cbs_id,player_owned_avg,player_owned_espn,player_owned_yahoo,player_image_url,player_square_image_url,rank_delta,ecr_type,scrape_date
1146193,/nfl/rankings/db.php,weekly-db,Budda Baker,16733,DB,ARI,1.00,0.00,1.0,1.0,budda baker,ARI,5ce20f3e-0f02-4f53-a2ef-5b076361f2b1,budda-baker.php,30149,2139625,64.8,65.5,64.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,NaN,wp,2020-10-16
1146194,/nfl/rankings/db.php,weekly-db,Landon Collins,13895,DB,WAS,2.67,0.75,2.0,10.0,landon collins,WAS,a9c41c5b-0dcf-40cc-a76c-644307f2f2df,landon-collins.php,28421,2000901,40.1,52.1,28.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,NaN,wp,2020-10-16
1146195,/nfl/rankings/db.php,weekly-db,Jeremy Chinn,19429,DB,CAR,4.00,1.15,3.0,11.0,jeremy chinn,CAR,5f623fbc-415e-4035-b423-7850cf1153b4,jeremy-chinn.php,32734,2247190,22.0,25.9,18.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,1.0,wp,2020-10-16
1146196,/nfl/rankings/db.php,weekly-db,Jordan Poyer,11759,DB,BUF,4.33,0.94,2.0,9.0,jordan poyer,BUF,95fab6fa-3ee1-47d0-93ad-c7ff41744be7,jordan-poyer.php,26841,1665374,49.5,47.9,51.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,-1.0,wp,2020-10-16
1146197,/nfl/rankings/db.php,weekly-db,John Johnson,16747,DB,LAR,5.83,1.95,2.0,15.0,john johnson,LAR,8c824157-eb33-4378-bf19-6c738a186ceb,john-johnson.php,30204,2082526,52.5,53.0,52.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,NaN,wp,2020-10-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197129,/nfl/rankings/db.php,weekly-db,K'Von Wallace,19319,DB,TEN,97.00,9.00,88.0,106.0,KVon Wallace,TEN,789af1aa-253e-4fda-a93b-cef346bd91b3,kvon-wallace.php,32797,2239538,1.7,2.3,1.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,-55.0,wp,2023-12-29
1197130,/nfl/rankings/db.php,weekly-db,Sean Murphy-Bunting,18648,DB,TEN,106.00,15.00,91.0,121.0,Sean Murphy-Bunting,TEN,f84bf885-d6ff-4fc8-8b6e-64bb3bceb17d,sean-bunting.php,31871,2188984,1.9,1.9,0.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,-17.0,wp,2023-12-29
1197131,/nfl/rankings/db.php,weekly-db,Jaylon Johnson,19254,DB,CHI,110.00,17.00,93.0,127.0,Jaylon Johnson,CHI,99b81b41-fb3b-4650-940b-9cb3770021ba,jaylon-johnson.php,32720,2827532,0.7,0.7,0.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,NaN,wp,2023-12-29
1197132,/nfl/rankings/db.php,weekly-db,Ronnie Harrison Jr.,17319,DB,IND,122.00,26.00,96.0,148.0,Ronnie Harrison,IND,9b96ef63-04bd-41ae-b59c-acc0f2561d19,ronnie-harrison.php,31063,2180568,0.2,0.2,0.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,NaN,wp,2023-12-29
